In [3]:
import requests
import pandas as pd
import numpy as np

def etherscan_request(action, 
                      address='0x7bea39867e4169dbe237d55c8242a8f2fcdcc387', 
                      api_key='Y8YP3J3BTG1DRZZPQ3F2I41E88I9GSUXNU', 
                      startblock=0, 
                      endblock=99999999, 
                      sort='asc'):
    
    base_url = 'https://api.etherscan.io/api'
    params = {
        'module': 'account',
        'action': action,
        'address': address,
        'startblock': startblock,
        'endblock': endblock,
        'sort': sort,
        'apikey': api_key
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    
    if data['status'] != '1':
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])

    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Set Transaction Value in Approrpraite Format
    df['og_value'] = df['value'].copy()
    df['value'] = np.where(df['tokenDecimal']=='6',df['value']/100000,df['value']/1000000000000000000)


    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': row['blockNumber'],
                'timeStamp': row['timeStamp'],
                'hash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': row['gas'],
                'gasPrice': row['gasPrice'],
                'gasUsed': row['gasUsed'],
                'cumulativeGasUsed': row['cumulativeGasUsed'],
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    return pd.DataFrame.from_dict(consolidated_data, orient='index'),df

consolidated,transactions = etherscan_request('tokentx')


In [4]:
consolidated

,blockNumber,timeStamp,hash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations
0x004cb88319b0678320cb0a04fab8003897f33c345576adfbb1f79b903a0509f0,12369811,1620158865,0x004cb88319b0678320cb0a04fab8003897f33c345576...,0xe79057088a15ac8c9c8bec3b90bd9891a1b3af51,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.279015,10000.00000,Wrapped Ether,USDC,5851485,49000000000,5368857,14242176,7968846
0x2d4763faa40fc2d6cdf7f8afeab5448c755ce327908546ce019ded583fab068f,12369971,1620160731,0x2d4763faa40fc2d6cdf7f8afeab5448c755ce3279085...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xe592427a0aece92de3edee1f18e0157c05861564,0.010000,334.65884,Wrapped Ether,USDC,307903,44950000000,237023,11762141,7968686
0x22ddba6cc2ec54322db54132340e7b73bf30807e1692fbdbd8c09cab380e792b,12370013,1620161213,0x22ddba6cc2ec54322db54132340e7b73bf30807e1692...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xe592427a0aece92de3edee1f18e0157c05861564,0.050000,1627.78264,Wrapped Ether,USDC,328763,58000000000,255413,8718964,7968644
0x79ce22cf42c24772cd3fe5875bea9bb38ffa55e340711fd42f2b8bbafeced9dc,12370189,1620163472,0x79ce22cf42c24772cd3fe5875bea9bb38ffa55e34071...,0xc148480a8eff5e6885ab0ac7905291dd11fd6499,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.022076,1000.00000,Wrapped Ether,USDC,543962,51000000000,451712,9534424,7968468
0xa503f71b5686bf337d46402a66abbe2bb97b230786c7a415701d5cc82e608746,12370268,1620164518,0xa503f71b5686bf337d46402a66abbe2bb97b230786c7...,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.200000,0.00000,Wrapped Ether,,527289,49000000000,451791,8933012,7968389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x0554a412c2f132016a4a759b366d5e14e564d2a5c040ba0e64fe65a99b17bf3e,12645022,1623840148,0x0554a412c2f132016a4a759b366d5e14e564d2a5c040...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,5.943004,148299.57000,Wrapped Ether,USDC,520188,10906117327,328988,11583794,7693635
0x5e8f4074ec552b11837b0698a029d58680de458b195c3f56cf0a1fcf4e55c6a2,12645026,1623840221,0x5e8f4074ec552b11837b0698a029d58680de458b195c...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xfd3dfb524b2da40c8a6d703c62be36b5d8540626,5.000000,124246.58905,Wrapped Ether,USDC,1739844,34000000000,1297382,4606515,7693631
0xeaee71f2d9a5104580961f83ff42cd97a95d67c367b0b4f1bd3f42d3fe3ec9ea,12645143,1623841643,0xeaee71f2d9a5104580961f83ff42cd97a95d67c367b0...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0x03d8c9b9f039d20abd957a416f6dc534577b164e,2.549273,63117.09081,Wrapped Ether,USDC,266384,18000000000,184131,6061687,7693514
0x57902cd448b9b99dbae2f6239fd37c564b5996c1f8ab69c15bb20d91b0d30a8d,12645174,1623842037,0x57902cd448b9b99dbae2f6239fd37c564b5996c1f8ab...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.374380,0.00000,Wrapped Ether,,285020,12000000000,192117,9926205,7693483


In [7]:
transactions

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,og_value
0,12369811,1620158865,0x004cb88319b0678320cb0a04fab8003897f33c345576...,192249,0x52dc59f33cf9f255fb533445f668221a9361914a626f...,0xe79057088a15ac8c9c8bec3b90bd9891a1b3af51,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,10000.000000,USDC,USDC,6,255,5851485,49000000000,5368857,14242176,deprecated,7968846,1.000000e+09
1,12369811,1620158865,0x004cb88319b0678320cb0a04fab8003897f33c345576...,192249,0x52dc59f33cf9f255fb533445f668221a9361914a626f...,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.279015,Wrapped Ether,WETH,18,255,5851485,49000000000,5368857,14242176,deprecated,7968846,2.790150e+17
2,12369971,1620160731,0x2d4763faa40fc2d6cdf7f8afeab5448c755ce3279085...,3,0x2a758995fd89a96765a34b1730f364cdeb03b799b235...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe592427a0aece92de3edee1f18e0157c05861564,334.658840,USDC,USDC,6,131,307903,44950000000,237023,11762141,deprecated,7968686,3.346588e+07
3,12369971,1620160731,0x2d4763faa40fc2d6cdf7f8afeab5448c755ce3279085...,3,0x2a758995fd89a96765a34b1730f364cdeb03b799b235...,0xe592427a0aece92de3edee1f18e0157c05861564,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.010000,Wrapped Ether,WETH,18,131,307903,44950000000,237023,11762141,deprecated,7968686,1.000000e+16
4,12370013,1620161213,0x22ddba6cc2ec54322db54132340e7b73bf30807e1692...,8,0xaae6432a6077b1de840c5035f618645cca424e786445...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe592427a0aece92de3edee1f18e0157c05861564,1627.782640,USDC,USDC,6,161,328763,58000000000,255413,8718964,deprecated,7968644,1.627783e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,12645026,1623840221,0x5e8f4074ec552b11837b0698a029d58680de458b195c...,14,0x23ea243bc91c660e573ca8abee7d441d0ac669652613...,0xfd3dfb524b2da40c8a6d703c62be36b5d8540626,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,5.000000,Wrapped Ether,WETH,18,26,1739844,34000000000,1297382,4606515,deprecated,7693631,5.000000e+18
9996,12645143,1623841643,0xeaee71f2d9a5104580961f83ff42cd97a95d67c367b0...,108,0xe87ddb19ef482357cfde28f139e037a4401847224b0b...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x03d8c9b9f039d20abd957a416f6dc534577b164e,63117.090810,USDC,USDC,6,107,266384,18000000000,184131,6061687,deprecated,7693514,6.311709e+09
9997,12645143,1623841643,0xeaee71f2d9a5104580961f83ff42cd97a95d67c367b0...,108,0xe87ddb19ef482357cfde28f139e037a4401847224b0b...,0xe592427a0aece92de3edee1f18e0157c05861564,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,2.549273,Wrapped Ether,WETH,18,107,266384,18000000000,184131,6061687,deprecated,7693514,2.549273e+18
9998,12645174,1623842037,0x57902cd448b9b99dbae2f6239fd37c564b5996c1f8ab...,113,0x43c931a2e033444e24d28b0f767dea5a2b3e8d6991a7...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.374380,Wrapped Ether,WETH,18,112,285020,12000000000,192117,9926205,deprecated,7693483,3.743797e+17
